# H5 generation - random sampling , normalization

## Dataset list
#### 20181020
* dataset/[db_aug_v2_20181019_ndb_aug_v2_20181020](https://drive.google.com/open?id=1mAEHXeOxb8jMM9mEy1bComWdxjk3cyCS)
* dataset/[db_aug_v2_v_050_20181019_ndb_aug_v2_20181020](https://drive.google.com/open?id=1n1c4dt_92ww0SUUKGZ-uZ5e0LDOAT9FO)
* dataset/[db_aug_v2_v_100_20181019_ndb_aug_v2_20181020](https://drive.google.com/open?id=1aH3MIDQ7WPn-BJVhORSRXvSKjFjC1DCm)
* dataset/[db_aug_v2_v_200_20181019_ndb_aug_v2_20181020](https://drive.google.com/open?id=1fn8XaLs7q60KXMXoMR9-4XcmpdGsfyPy)
* dataset/[db_aug_v2_v_500_20181019_ndb_aug_v2_20181020](https://drive.google.com/open?id=1vG1FQt54Mgc7IkR0i-p6b_4Og9Zit0g7)
#### 20181022
* dataset/[db_aug_v2_v_100_20181022_ndb_aug_v2_20181020](https://drive.google.com/open?id=1eLiTpjTuPnwWsJEIKR9jdDmbDArcfuhp)
* dataset/[db_aug_v2_v_200_20181022_ndb_aug_v2_20181020](https://drive.google.com/open?id=1MnXCirEozU2FVm2D0Xx7wCFYbeY8KHv3)
* dataset/[db_aug_v2_v_300_20181022_ndb_aug_v2_20181020](https://drive.google.com/open?id=1gWS7D3KI-i6l3t9JLanI6BppPwviW_Hs)
* dataset/[db_aug_v2_v_400_20181022_ndb_aug_v2_20181020](https://drive.google.com/open?id=11PiIsQfoKZhikkJb8wVor2g_UZQERxCv)
* dataset/[db_aug_v2_v_500_20181022_ndb_aug_v2_20181020](https://drive.google.com/open?id=1Vo9BOXSeqXs-Zi30eWJ-fXDzog3Me1HL)

In [21]:
import numpy as np

import pickle

import os, sys

import h5py

import os, sys

from tqdm import tqdm, trange

sys.path.append( os.path.abspath('../scripts') )
from utils import *


In [17]:
num_sample = 256
num_random_sample = 10


In [18]:
""" load data and resampling """

object_list = []
object_list.append('world_RangeRoverSportSVRSimple')
object_list.append('world_BmwX5Simple')
object_list.append('world_CitroenCZeroSimple')
object_list.append('world_LincolnMKZSimple')
object_list.append('world_ToyotaPriusSimple')
object_list.append('world_BusSimple')
object_list.append('world_TruckSimple')

object_label_list = [1,1,1,1,1, 2,2]

"""
unkown: 0
cars: 1
trucks: 2
""" 

## Load positive data
# load_data_path = os.path.abspath('../dataset/db_aug_v2_20181019.pickle')
# load_data_path = os.path.abspath('../dataset/db_aug_v2_v_500_20181019.pickle')
load_data_path = os.path.abspath('../dataset/db_aug_v2_v_500_20181022.pickle')

with open(load_data_path, 'rb') as f:
    db_aug_test = pickle.load(f)   
    
## Load negative data
load_ndata_path = os.path.abspath('../dataset/ndb_aug_v2_20181020.pickle')
with open(load_ndata_path, 'rb') as f:
    ndb = pickle.load(f)   


In [19]:
""" data / label parsing from db """

## Positive data
data = []
label = []

for i in range(len(object_list)):
    
    key = object_list[i]
    cls = object_label_list[i]
    
    print key, cls
    sys.stdout.flush()
    
    i = 0
    for cloud in tqdm(db_aug_test[key]):
        
        for _ in range(num_random_sample): 
            
            data.append(NormalizeResample(cloud, num_sample))
            label.append(cls)
            
sys.stdout.flush()
print "Only positive:"
print "data:", len(data)
print "label:", len(label)
sys.stdout.flush()

## Negative data
for i in trange(  len( ndb['unknown'] )  ):
    
    cloud = ndb['unknown'][i]
    cls = 0
        
    for _ in range(num_random_sample): 

        data.append(NormalizeResample(cloud, num_sample))
        label.append(cls)

sys.stdout.flush()
print "With negative:"
print "data:", len(data)
print "label:", len(label)
sys.stdout.flush()

world_RangeRoverSportSVRSimple 1


100%|██████████| 480/480 [00:00<00:00, 1827.67it/s]

world_BmwX5Simple 1



100%|██████████| 480/480 [00:00<00:00, 2252.27it/s]

world_CitroenCZeroSimple 1


...

world_TruckSimple 2



100%|██████████| 480/480 [00:00<00:00, 2256.80it/s]

Only positive:
data: 33600
label: 33600



100%|██████████| 3948/3948 [00:01<00:00, 2008.05it/s]

With negative:
data: 73080
label: 73080


In [20]:
out_path = load_data_path.split('.')[0] + "_" + load_ndata_path.split('/')[-1].split('.')[0]
if not os.path.exists(out_path):
    os.mkdir(out_path)

## save h5 file    
print "load_data_path:", load_data_path
print "load_ndata_path:", load_ndata_path
print "out_path:", out_path
save_h5_files(data, label, out_path, data_dtype, label_dtype)

load_data_path: /media/kyungpyo/SmartCarContest/MasterThesisProject/Dataset/dataset/db_aug_v2_v_500_20181022.pickle
load_ndata_path: /media/kyungpyo/SmartCarContest/MasterThesisProject/Dataset/dataset/ndb_aug_v2_20181020.pickle
out_path: /media/kyungpyo/SmartCarContest/MasterThesisProject/Dataset/dataset/db_aug_v2_v_500_20181022_ndb_aug_v2_20181020
Total data: 73080
Total label: 73080
0
np_data_filtered.shape (39480, 256, 3)
data_train.shape (23688, 256, 3)
data_test.shape (7896, 256, 3)
data_vali.shape (7896, 256, 3)
1
np_data_filtered.shape (24000, 256, 3)
data_train.shape (14400, 256, 3)
data_test.shape (4800, 256, 3)
data_vali.shape (4800, 256, 3)
2
np_data_filtered.shape (9600, 256, 3)
data_train.shape (5760, 256, 3)
data_test.shape (1920, 256, 3)
data_vali.shape (1920, 256, 3)
[Generate] path: /media/kyungpyo/SmartCarContest/MasterThesisProject/Dataset/dataset/db_aug_v2_v_500_20181022_ndb_aug_v2_20181020/train.h5, 
data shape: (43848, 256, 3), label shape: (43848,)
[Generate] pat